# FlowRec

In [ ]:
!pip install scikit-multiflow

In [ ]:
!wget -q --show-progress https://github.com/paraschakis/flowrec/raw/master/data/trivago_1M100K.csv
# !wget -q --show-progress https://github.com/paraschakis/flowrec/raw/master/data/yoochoose_clicks_1M100K.csv
# !wget -q --show-progress https://github.com/paraschakis/flowrec/raw/master/data/clef_1M100K.csv

trivago_1M100K.csv. 100%[===================>]  26.49M   130MB/s    in 0.2s    


In [ ]:
# !git clone https://github.com/paraschakis/flowrec.git
# %cd flowrec
# import sys
# sys.path.append('.')

In [ ]:
from skmultiflow.data import FileStream

In [ ]:
from evaluation.evaluate_prequential import EvaluatePrequential
from recommendation.random import RandomClassifier
from recommendation.popular import PopularClassifier
from recommendation.co_events import CoEventsClassifier
from recommendation.seq_events import SeqEventsClassifier
from recommendation.ht_wrapper import HTWrapper
from recommendation.beer import BeerEnsemble
from recommendation.sknn import SKNNClassifier

In [ ]:
# Create stream
stream = FileStream("trivago_1M100K.csv")
stream.prepare_for_use()

In [ ]:
# Instantiate recommenders
random = RandomClassifier()
ht = HTWrapper(weight_mc=5, weight_inv=0.90)
sknn = SKNNClassifier(k=100, sample_size=1000, sample_recent=True,
                      similarity='cosine', sliding_window=True)

popular = PopularClassifier(sliding_window=True)
ar = CoEventsClassifier(sliding_window=False)
sr = SeqEventsClassifier(sliding_window=False)
mc = SeqEventsClassifier(steps_back=1, sliding_window=False)
beer = BeerEnsemble(cf_components=[ar, sr, mc, popular, sknn])

evaluator = EvaluatePrequential(session_column_index=0,
                                time_column_index=1,
                                rec_size=10,
                                allow_reminders=True,
                                allow_repeated=True,
                                show_plot=False,
                                n_wait=100,
                                n_keep=500,
                                n_skip=1000,
                                pretrain_size=0,
                                max_samples=10000,
                                metrics=['recall', 'mrr', 'running_time'])

# Run prequential evaluation
evaluator.evaluate(stream=stream, model=[ar, sr, mc, popular, random, sknn, beer, ht],
                   model_names=['AR', 'SR', 'MC', 'POP', 'RAND', 'S-KNN', 'BEER[TS]', 'HT'])

Prequential Evaluation
Evaluating 1 target(s).
Evaluating...
 #################### [100%] [551.30s]
Processed samples: 10000
Mean performance:
AR - Recall@10    : 0.0267
AR - Mrr@10    : 0.0232
AR - Training time (s)  : 0.86
AR - Testing time  (s)  : 6.86
AR - Total time    (s)  : 7.72
SR - Recall@10    : 0.0075
SR - Mrr@10    : 0.0060
SR - Training time (s)  : 0.62
SR - Testing time  (s)  : 5.29
SR - Total time    (s)  : 5.91
MC - Recall@10    : 0.0062
MC - Mrr@10    : 0.0053
MC - Training time (s)  : 0.52
MC - Testing time  (s)  : 5.12
MC - Total time    (s)  : 5.63
POP - Recall@10    : 0.0020
POP - Mrr@10    : 0.0006
POP - Training time (s)  : 0.23
POP - Testing time  (s)  : 12.01
POP - Total time    (s)  : 12.25
RAND - Recall@10    : 0.0000
RAND - Mrr@10    : 0.0000
RAND - Training time (s)  : 0.02
RAND - Testing time  (s)  : 0.97
RAND - Total time    (s)  : 0.99
S-KNN - Recall@10    : 0.0127
S-KNN - Mrr@10    : 0.0123
S-KNN - Training time (s)  : 0.99
S-KNN - Testing time  (s)  : 

[CoEventsClassifier(sliding_window=False, target_event_type=None),
 SeqEventsClassifier(sliding_window=False, source_event_type=None,
                     steps_back=inf, target_event_type=None),
 SeqEventsClassifier(sliding_window=False, source_event_type=None, steps_back=1,
                     target_event_type=None),
 PopularClassifier(event_type=None, sliding_window=True),
 RandomClassifier(),
 SKNNClassifier(k=100, sample_recent=True, sample_size=1000, similarity='cosine',
                sliding_window=True),
 BeerEnsemble(attr_file=None, boundaries=[],
              cf_components=[CoEventsClassifier(sliding_window=False,
                                                target_event_type=None),
                             SeqEventsClassifier(sliding_window=False,
                                                 source_event_type=None,
                                                 steps_back=inf,
                                                 target_event_type=None),
       